In [7]:
# Packages
import socket
import pickle
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

In [ ]:
# Load the dataframes dictionary from the file
with open("../dataframes/th_mhwt_dataframes_dict.pkl", "rb") as file:
    dataframes = pickle.load(file)

   year  threshold       mhwt ensemble
0  1980  29.254530  30.587918       01
1  1982  29.311581  30.649778       01
2  1984  29.371500  30.713617       01
3  1986  29.434034  30.779874       01
4  1988  29.498776  30.849119       01


In [9]:
# Function to find a free port
def get_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))  # Let the OS choose a free port
        return s.getsockname()[1]

# Create Dash app
#app = dash.Dash(__name__)
app = Dash(__name__)

app.layout = html.Div([
    html.H1("City Heatwave Metrics by Ensemble"),

    html.Div([
        html.Label("Select City:"),
        dcc.Dropdown(
            id="city-dropdown",
            options=[{"label": city.replace("_", " "), "value": city} for city in dataframes],
            value=list(dataframes.keys())[0]
        )
    ], style={"width": "48%", "display": "inline-block"}),

    html.Div([
        html.Label("Select Variable:"),
        dcc.Dropdown(
            id="variable-dropdown",
            options=[
                {"label": "Threshold", "value": "threshold"},
                {"label": "Mean Heatwave Temp", "value": "mhwt"}
            ],
            value="threshold"
        )
    ], style={"width": "48%", "display": "inline-block", "float": "right"}),

    dcc.Graph(id="city-plot")
])

@app.callback(
    Output("city-plot", "figure"),
    Input("city-dropdown", "value"),
    Input("variable-dropdown", "value")
)
def update_plot(selected_city, selected_variable):
    df = dataframes[selected_city]
    fig = px.line(
        df,
        x="year",
        y=selected_variable,
        color="ensemble",
        title=f"{selected_variable.upper()} over Time for {selected_city.replace('_', ' ')} by Ensemble"
    )

    for trace in fig.data:
        if trace.name == "mean":
            trace.line.color = "black"
            trace.line.dash = "dash"
    fig.update_layout(yaxis_title=selected_variable.upper())
    return fig

if __name__ == "__main__":
    port = get_free_port()
    print(f"App running on http://127.0.0.1:{port}")
    app.run(debug=True, port=port)


App running on http://127.0.0.1:50623
